### Concept
The notebook simply uses **neural network algorithms (MLP)** to estimate indoor locations. It doesn't have high performance, but it was quite an interesting attempt. I've reinvented the wheel.

We used [**Wi-Fi signal feature dataset**](https://www.kaggle.com/hiro5299834/indoor-navigation-and-location-wifi-features) shared by [**BIZEN**](https://www.kaggle.com/hiro5299834). We appreciate his sharing!! <br/>
Please adjust the parameters according to GPU performance. Then you can get better results.

### Network settings
Network Architecture: input size - input size/2 - input size/20 - 1<br/>
Activation Function : Sigmoid & Relu| Linear Regression <br/>
Optimizers : Adam | MSE <br/>
Batch size : 32 <br/>



In [ ]:
#%% Import Libraries

import os
import glob
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

#%% Test & Train File path defintion

train_dir = sorted(glob.glob('../input/indoor-navigation-and-location-wifi-features/*_train.csv'))
test_dir = sorted(glob.glob('../input/indoor-navigation-and-location-wifi-features/*_test.csv'))

final_ver = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')

submission = []
k =[[0,0,0]]

#%% TRAIN
for step in range(24):
    output = []
    
    # load data
    train_data = pd.read_csv(train_dir[step],index_col=0)
    train_data = train_data.drop(['path'],axis=1)
    
    # test
    test_data = pd.read_csv(test_dir[step],index_col=0)
    test_data = test_data.iloc[:,:-1]
    
    # Normalization [0 ~ 1]
    maxn = train_data.max()
    minn = train_data.min()
    
    train_data = (train_data - minn) / (maxn - minn)
    
    maxt = test_data.max()
    mint = test_data.min()
    
    test_data = (test_data - mint) / (maxt - mint)
    test_data[test_data.isna()] = 0.0
    
    # Shuffle
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    num = int(train_data.shape[0] * 0.90)
    
   
    # Input dataset
    train_in = train_data.iloc[:num]
    valid_in = train_data.iloc[num:]
    
    train_ins = train_in.iloc[:,:-3]
    valid_ins = valid_in.iloc[:,:-3]
    
    
    results = []
    predictions = []
    print('STEP {}: {} Training Stage Start'.format(step,os.path.basename(train_dir[step])))
    
    for stage in range(3):
        st = ['X','Y','F']
        train_ous = train_in.iloc[:,stage-3]
        valid_ous = valid_in.iloc[:,stage-3]
    
        # Network modeling
        im_size = train_ins.shape[1]
        w1 = int(im_size/2)
        w2 = int(im_size/20)
    
        model = keras.Sequential([
            keras.layers.Flatten(input_shape=(im_size, 1)),
            keras.layers.Dense(w1, activation='sigmoid'),
            keras.layers.Dense(w2, activation='sigmoid'),
            keras.layers.Dense(1, activation='sigmoid') 
        ])
    
        model.compile(optimizer= tf.keras.optimizers.Adam(
                        learning_rate=0.0005, beta_1=0.9, 
                        beta_2=0.999, epsilon=1e-07, 
                        amsgrad=False,name='Adam'),
                        loss='mse',metrics=[ 'mse'])
    
        print('STEP {}-{}: ({}) Training {} Stage...'.format(step,stage,im_size,st[stage]))
    
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience = 10)
        history = model.fit(train_ins, train_ous ,batch_size=32,
                            validation_data=(valid_ins, valid_ous),
                            epochs = 100,callbacks=[callback],
                            verbose = 0)
    
        
        prediction = model.predict(valid_ins)
        prediction = prediction * (maxn[stage-3] - minn[stage-3]) + minn[stage-3]
        target = valid_ous * (maxn[stage-3] - minn[stage-3]) + minn[stage-3]
        
        
        if stage < 2:
            mse = np.mean(np.sqrt(np.power(prediction.T - np.array(target),2)))
            results.append(np.power(prediction.T - np.array(target),2))
            print('{} Position MSE: {}'.format(st[stage],mse))
        else:
            prediction = np.round(prediction)
            mse = np.mean(15 * (np.abs(prediction.T - np.array(target))))
            print('{} Position MSE: {}'.format(st[stage],mse))
            
            mpe = np.mean(np.sqrt(results[0] + results[1]) + mse)
            print('Evaluation Results MPE : {}\n'.format(mpe))
        
        prediction = model.predict(test_data)  
        prediction = prediction * (maxn[stage-3] - minn[stage-3]) + minn[stage-3]
        if stage == 2: prediction = np.round(prediction)
        predictions.append(prediction)
    
    submission.append(predictions)
    
for i in submission:
    k = np.vstack((np.array(k),np.hstack((i[2],i[0],i[1]))))
    
final_ver.loc[:,'floor':] = k[1:]
final_ver.to_csv('./submission.csv', index = False)